In [1]:
import os

def find_project_root(start_dir="."):
    current_dir = os.path.abspath(start_dir)
    while True:
        if os.path.exists(os.path.join(current_dir, ".project_root")):
            return current_dir
        
        parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
        
        if parent_dir == current_dir:
            raise FileNotFoundError("Project root not found.")
        
        current_dir = parent_dir

project_root = find_project_root()
os.chdir(project_root)
print("Current working directory:", os.getcwd())


Current working directory: c:\Users\Wiede\OneDrive - Queen Mary, University of London\Documentation\Projects\HCockpit


## Load Model

In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import dotenv

dotenv.load_dotenv()

llm = ChatOpenAI(model="gpt-4o")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

## Single Chat

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

output_parser = StrOutputParser()
chain = prompt | llm | output_parser

chain.invoke({"input": "Hi!"})

'Hello! How can I assist you today?'

Failed to get info from https://api.smith.langchain.com: ConnectionError(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)))


## Multimodal

In [4]:
import base64

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')
  
image_base64 = encode_image("./docs/figures/s-3.png")

In [6]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What is in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/png;base64,{image_base64}",
            "detail": "auto"
          }
        }
      ]
    }
  ],
  max_tokens=300
)

print(response.choices[0].message.content)

The image illustrates an automotive AI system designed to assist a driver. Below is a breakdown of the elements present in the image:

1. **Center Image (Driver's View)**: This shows a simulated first-person perspective from inside a car. The driver's hand is on the steering wheel, and a sun glare is visible, suggesting visibility challenges. There is an icon that suggests the system is monitoring the surroundings.

2. **AI Process Flow**:
    - **H.Copilot**: Depicted as a cartoonish figure wearing goggles, signifying the AI 'co-pilot'.
    - **Sensors and Tools**: Labeled lines indicate tools and sensors feeding information into the H.Copilot system:
        - `tools.obs_sensor()`
        - `tools.obs_hctrl()`
        - `tools.obs_hctr()`
        - `tools.set_cam_to()`
        - `tools.speech()`

3. **Right Image (External View)**: This shows a streetscape with a car approaching from the left, implying a potential hazard for the driver.

4. **Output Panel**:
    - **Perception**: Des

In [7]:
from langchain_core.prompts.image import ImagePromptTemplate

messages = [
    {"type": "text", "text": "{input}"},
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png;base64,{image_base64}",
            # "url": "{img_url}",
            "detail": "auto"
        }
    }
]

prompt_openai = [
    {
        "role": "user",
        "content": messages
    }
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical expert."),
    HumanMessage(content=messages),
])

chain = prompt | llm | output_parser
chain.invoke({"input": "What is in this image?"})
# chain.invoke({"input": "What is in this image?",
#               "img_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
#               })

'This image appears to describe a scenario involving an autonomous driving system, likely guided by some sort of AI co-pilot, here referred to as "HCopilot". The process can be summarized as follows:\n\n1. **Observation**:\n   - The system uses sensors (tools.obs_sensor) to observe the surroundings. In this instance, it detects:\n     - A car approaching from the left.\n     - The driver is not paying attention to the approaching car.\n     - The driver\'s intention to go straight.\n     - The sun is causing dazzling light, which could impair visibility.\n\n2. **Perception**:\n   - The perception module of HCopilot interprets the sensor data and infers the current scenario:\n     - A car is coming from the left.\n     - The driver is not attentive to the approaching car.\n     - The driver intends to drive straight.\n     - The sun\'s glare adds to the challenge of driving safely.\n\n3. **Planning**:\n   - Based on the perception, HCopilot formulates a plan.\n   - The plan suggests tha

## RAG Demo

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.image.ImagePromptTemplate.html")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

vector = FAISS.from_documents(documents, embeddings)

In [7]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

retriever = vector.as_retriever()
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

input = "Does LangChain accept image prompt?"
res = retrieval_chain.invoke({"input": input})

print(res["answer"])

Yes, LangChain accepts image prompts. The `ImagePromptTemplate` class mentioned in the context is designed for creating prompts for a multimodal model with image inputs. This template allows the creation of new models by parsing and validating input data specifically designed to handle ImageURLs, confirming that LangChain can work with image-based inputs.


## Multi-Round Chat Demo

In [8]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Does LangChain accept image prompt?"), AIMessage(content="Yes!")]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='langchain_core.prompts.image.ImagePromptTemplate — 🦜🔗 LangChain 0.1.17\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangChain\n\n\nCore\n\n\nCommunity\n\n\nExperimental\n\n\nText splitters\n\n\nai21\n\n\nairbyte\n\n\nanthropic\n\n\nastradb\n\n\nchroma\n\n\ncohere\n\n\nelasticsearch\n\n\nexa\n\n\nfireworks\n\n\ngoogle-genai\n\n\ngoogle-vertexai\n\n\ngroq\n\n\nibm\n\n\nmistralai\n\n\nmongodb\n\n\nnomic\n\n\nnvidia-ai-endpoints\n\n\nnvidia-trt\n\n\nopenai\n\n\npinecone\n\n\npostgres\n\n\nrobocorp\n\n\ntogether\n\n\nupstage\n\n\nvoyageai\n\n\nPartner libs\n\nai21\nairbyte\nanthropic\nastradb\nchroma\ncohere\nelasticsearch\nexa\nfireworks\ngoogle-genai\ngoogle-vertexai\ngroq\nibm\nmistralai\nmongodb\nnomic\nnvidia-ai-endpoints\nnvidia-trt\nopenai\npinecone\npostgres\nrobocorp\ntogether\nupstage\nvoyageai\n\n\n\nDocs\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nToggle Menu\n\n\n\nPrev\nUp\nNext\n\n\n\nlangchain_core.prompts.image.ImagePromptTemplate\nImagePromptTemplate\nImagePromp

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [10]:
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how with example"
})

{'chat_history': [HumanMessage(content='Does LangChain accept image prompt?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how with example',
 'context': [Document(page_content='langchain_core.prompts.image.ImagePromptTemplate — 🦜🔗 LangChain 0.1.17\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangChain\n\n\nCore\n\n\nCommunity\n\n\nExperimental\n\n\nText splitters\n\n\nai21\n\n\nairbyte\n\n\nanthropic\n\n\nastradb\n\n\nchroma\n\n\ncohere\n\n\nelasticsearch\n\n\nexa\n\n\nfireworks\n\n\ngoogle-genai\n\n\ngoogle-vertexai\n\n\ngroq\n\n\nibm\n\n\nmistralai\n\n\nmongodb\n\n\nnomic\n\n\nnvidia-ai-endpoints\n\n\nnvidia-trt\n\n\nopenai\n\n\npinecone\n\n\npostgres\n\n\nrobocorp\n\n\ntogether\n\n\nupstage\n\n\nvoyageai\n\n\nPartner libs\n\nai21\nairbyte\nanthropic\nastradb\nchroma\ncohere\nelasticsearch\nexa\nfireworks\ngoogle-genai\ngoogle-vertexai\ngroq\nibm\nmistralai\nmongodb\nnomic\nnvidia-ai-endpoints\nnvidia-trt\nopenai\npinecone\npostgres\nrobocorp\ntogether\nupstage\nvoyageai\n\n\n\

## Agent

In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [12]:
from langchain_core.messages import AIMessage
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

tools = []

def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls